In [36]:
import re
import requests
from bs4 import BeautifulSoup
import sqlite3
from pymystem3 import Mystem
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from thisapidoesnotexist import get_cat
from matplotlib import pyplot as plt

conn = sqlite3.connect('stihi.db')
c = conn.cursor()
m = Mystem()
c.execute("CREATE TABLE IF NOT EXISTS 'Stihi'('id' integer PRIMARY KEY AUTOINCREMENT, 'word1', 'lemma1', 'pos1', 'word2', 'lemma2', 'pos2', 'word3', 'lemma3', 'pos3', 'texts' text, 'url')")

Функция, достающая стихи про животных с сайта Rustih.ru

In [40]:
def get_texts():
    texts = []
    hrefs = []
    response = requests.get('https://rustih.ru/stixi-o-zhivotnyx/', headers={'User-agent':'Mozilla/5.0'}).text
    soup = BeautifulSoup(response,'html.parser')
    et = soup.find_all('div', {'class': 'entry-title'})
    for i in et:
        hr = i.find('a').attrs['href']
        hrefs.append(hr)
        response = requests.get(hr, headers={'User-agent':'Mozilla/5.0'}).text
        soup = BeautifulSoup(response,'html.parser')
        pt = soup.find('div', {'class': 'entry-content poem-text'})
        text = pt.find_all(['p', 'h2'])
        t = ''
        for j in text:
            if 'adsbygoogle' not in j.text:
                if 'I' not in j.text:
                    j = re.sub('<br/>', '\n', str(j))
                    j = re.sub('</p>', '\n', j)
                    j = re.sub('<p>', '', j)
                    clt = re.sub('\\u200e', '', j)
                    t += clt + '\n'
            else:
                break
        texts.append(t)

    return texts, hrefs

In [35]:
response = requests.get('https://rustih.ru/nikolaj-nekrasov-dedushka-mazaj-i-zajcy/').text
soup = BeautifulSoup(response,'html.parser')
pt = soup.find('div', {'class': 'entry-content poem-text'})
text = pt.find_all(['p', 'h2'])
t = ''
for j in text:
    if 'adsbygoogle' not in j.text:
        j = re.sub('<br/>', '\n', str(j))
        j = re.sub('</p>', '\n', j)
        j = re.sub('<p>', '', j)
        clt = re.sub('\\u200e', '', j)
        t += clt + '\n'
    else:
        break
print(t)

I

В августе, около Малых Вежей,
 С старым Мазаем я бил дупелей.
 Как-то особенно тихо вдруг стало,
 На́ небе солнце сквозь тучу играло.
 Тучка была небольшая на нем,
 А разразилась жестоким дождем!
 Прямы и светлы, как прутья стальные,
 В землю вонзались струи дождевые
 С силой стремительной… Я и Мазай,
 Мокрые, скрылись в какой-то сарай.
 Дети, я вам расскажу про Мазая.
 Каждое лето домой приезжая,
 Я по неделе гощу у него.
 Нравится мне деревенька его:
 Летом ее убирая красиво,
 Исстари хмель в ней родится на диво,
 Вся она тонет в зеленых садах;
 Домики в ней на высоких столбах
 (Всю эту местность вода понимает,
 Так что деревня весною всплывает,
 Словно Венеция). Старый Мазай
 Любит до страсти свой низменный край.
 Вдов он, бездетен, имеет лишь внука,
 Торной дорогой ходить ему — скука!
 За́ сорок верст в Кострому прямиком
 Сбегать лесами ему нипочем:
 «Лес не дорога: по птице, по зверю
 Выпалить можно». — А леший? — «Не верю!
 Раз в кураже я их звал-поджидал
 Целую ночь, — никого

In [41]:
texts, hrefs = get_texts()

Функция, заносящая тексты в базу в нужном формате.
Колонки в таблице: word1, lemma1, pos1, word2, lemma2, pos2, word3, lemma3, pos3, texts, url

In [42]:
def make_db(texts, hrefs):
    for i in tqdm(range(len(texts))):
        sents = sent_tokenize(texts[i])
        for s in sents:
            ana = []
            for w in m.analyze(s):
                if 'analysis' in w:
                    if w['analysis'] == []:
                        pos = ''
                        lemma = ''
                    elif 'gr' not in w['analysis'][0]:
                        pos = ''
                    elif 'lex' not in w['analysis'][0]:
                        lemma = ''
                    else:
                        gr = w['analysis'][0]['gr']
                        pos = gr.split('=')[0].split(',')[0]
                        lemma = w['analysis'][0]['lex']
                    ana.append((w['text'].lower(), lemma, pos))
            ana.extend([('', '', ''), ('', '', '')])
                    
            for j in range(len(ana[:-2])):
                v = (ana[j][0], ana[j][1], ana[j][2], ana[j+1][0], ana[j+1][1], ana[j+1][2], ana[j+2][0], ana[j+2][1], ana[j+2][2], s, hrefs[i])
                c.execute("INSERT INTO Stihi (word1, lemma1, pos1, word2, lemma2, pos2, word3, lemma3, pos3, texts, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", v)
                conn.commit()


In [43]:
make_db(texts, hrefs)

100%|██████████| 115/115 [04:29<00:00,  2.34s/it]


In [44]:
conn.close()

In [46]:
print(texts[5])

Рвусь из сил — и из всех сухожилий,
 Но сегодня — опять как вчера:
 Обложили меня, обложили —
 Гонят весело на номера!

Из-за елей хлопочут двустволки —
 Там охотники прячутся в тень, —
 На снегу кувыркаются волки,
 Превратившись в живую мишень.

Идёт охота на волков,
 Идёт охота —
 На серых хищников
 Матёрых и щенков!
 Кричат загонщики, и лают псы до рвоты,
 Кровь на снегу — и пятна красные флажков.

Не на равных играют с волками
 Егеря, но не дрогнет рука:
 Оградив нам свободу флажками,
 Бьют уверенно, наверняка.

Волк не может нарушить традиций —
 Видно, в детстве, слепые щенки,
 Мы, волчата, сосали волчицу
 И всосали: нельзя за флажки!

И вот — охота на волков,
 Идёт охота —
 На серых хищников
 Матёрых и щенков!
 Кричат загонщики, и лают псы до рвоты,
 Кровь на снегу — и пятна красные флажков.

Наши ноги и челюсти быстры —
 Почему же — вожак, дай ответ —
 Мы затравленно мчимся на выстрел
 И не пробуем через запрет?!

Волк не может, не должен иначе.
 Вот кончается время моё:
 Тот, к